See Kim, Kim, Shin 1995 (A general ...), section 4

In [ ]:
from IPython.display import HTML
import numpy as np

In [ ]:
from helper import angles2quat, animate_rotations
from unit_quaternion import UnitQuaternion

In [ ]:
def alpha(i, t):
    if t < i:
        return 0
    elif t >= i + 1:
        return 1
    else:
        return t - i

In [ ]:
# NB: math.prod() since Python 3.8
product = np.multiply.reduce

In [ ]:
def make_piecewise_slerp(qs):
    q0 = qs[0]
    omegas = [
        np.asarray((q_i_1.inverse() * q_i).log_map())
        for q_i_1, q_i in zip(qs, qs[1:])]
    
    def func(t):
        return q0 * product([
            UnitQuaternion.exp_map(omega * alpha(i, t))
            for i, omega in enumerate(omegas)
        ])
    
    return func

In [ ]:
qs = [
    angles2quat(0, 0, 0),
    angles2quat(90, 0, 0),
    angles2quat(90, 90, 0),
    angles2quat(90, 90, 90),
]

In [ ]:
times = np.linspace(0, len(qs) - 1, 100)

In [ ]:
piecewise_slerp = make_piecewise_slerp(qs)

In [ ]:
rotations = [piecewise_slerp(t) for t in times]

In [ ]:
ani = animate_rotations(rotations, interval=30)

In [ ]:
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
def _bernstein_bases(degree, t):
    return [
        _comb(degree, i) * t**i * (1 - t)**(degree - i)
        for i in range(degree + 1)]

from math import factorial as _factorial

def _comb(n, k):
    # NB: Python 3.8 has math.comb()
    return _factorial(n) // _factorial(k) // _factorial(n - k)

In [ ]:
_bernstein_bases(3, 0.7)

In [ ]:
from itertools import accumulate

In [ ]:
def cumulative_bases(degree, t):
    return list(accumulate(_bernstein_bases(degree, t)[::-1]))[::-1]


In [ ]:
cumulative_bases(3, 0.7)

In [ ]:
def make_bezier(qs):
    q0 = qs[0]
    omegas = [
        np.asarray((q_i_1.inverse() * q_i).log_map())
        for q_i_1, q_i in zip(qs, qs[1:])]
    degree = len(omegas)
    
    def func(t):
        b0, *bases = cumulative_bases(degree, t)
        assert np.isclose(b0, 1)
        return q0 * product([
            UnitQuaternion.exp_map(omega * basis)
            for omega, basis in zip(omegas, bases)
        ])
    
    return func

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
bezier = make_bezier(qs)

In [ ]:
rotations = [bezier(t) for t in times]

In [ ]:
ani = animate_rotations(rotations, interval=30)

In [ ]:
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
qs = [
    angles2quat(-45, 0, 0),
    angles2quat(0, 90, 0),
    angles2quat(0, -90, 0),
    angles2quat(45, 0, 0),
]

In [ ]:
bezier = make_bezier(qs)

In [ ]:
rotations = [bezier(t) for t in times]

In [ ]:
ani = animate_rotations(rotations, interval=30)

In [ ]:
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
from unit_quaternion import BezierSpline

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
control_polygons = [
    (
        angles2quat(i, 0, 0),
        angles2quat(i + 45, 90, 0),
        angles2quat(i + 45, -90, 0),
        angles2quat(i + 90, 0, 0),
    )
    for i in [0, 90, 180, 270]
]

In [ ]:
cumulative = []
for polygon in control_polygons:
    bezier = make_bezier(polygon)
    cumulative.extend(bezier(t) for t in times[:-1])

In [ ]:
cumulative_reversed = []
for polygon in control_polygons[::-1]:
    bezier = make_bezier(polygon[::-1])
    cumulative_reversed.extend(bezier(t) for t in times[1:])
cumulative_reversed = cumulative_reversed[::-1]

In [ ]:
casteljau = [
    rotation
    for polygon in control_polygons
    for rotation in BezierSpline([polygon]).evaluate(times[:-1])]

In [ ]:
casteljau_reversed = [
    rotation
    for polygon in control_polygons[::-1]
    for rotation in BezierSpline([polygon[::-1]]).evaluate(times[1:])][::-1]

In [ ]:
for one, two in zip(casteljau, casteljau_reversed):
    assert np.isclose(one.scalar, two.scalar)
    assert np.isclose(one.vector[0], two.vector[0])
    assert np.isclose(one.vector[1], two.vector[1])
    assert np.isclose(one.vector[2], two.vector[2])

In [ ]:
ani = animate_rotations({
    'De Casteljau': casteljau,
    'Cumulative': cumulative,
    'Cumulative reversed': cumulative_reversed,
}, figsize=(9, 3), interval=30)

In [ ]:
display(HTML(ani.to_jshtml(default_mode='loop')))